In [1]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#  install.packages("BiocManager")
#BiocManager::install("iterativeBMA")
#install.packages("PDtoolkit")
# install.packages("remotes")
# remotes::install_github("ayhandis/creditR")

##########
# There doesn't appear to be an R implementation of the Spiegelhalter test.
# The SAS implementation seems to be very limited in its scope: it doesn't let you
# specify the hypothesized success probabilities as far as I can tell.

In [2]:
library(PDtoolkit, quietly=T)

options(warn=-1)

Warning message:
"replacing previous import 'lifecycle::last_warnings' by 'rlang::last_warnings' when loading 'tibble'"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'Hmisc'


The following objects are masked from 'package:dplyr':

    src, summarize


The following objects are masked from 'package:base':

    format.pval, units



Attaching package: 'PDtoolkit'


The following object is masked from 'package:stats':

    power




In [3]:
# Import data
test_data_main <- read.csv("pd_test_data_main.csv")
test_data_period_2 <- read.csv("pd_test_data_period_2.csv")

In [4]:
# Aggregate the results
bin_data <- test_data_main %>%
            group_by(ratings) %>%
            summarise_at(vars(predicted_pd, default_flag), funs(mean(.), sum(.), n()))

bin_data

ratings,predicted_pd_mean,default_flag_mean,predicted_pd_sum,default_flag_sum,predicted_pd_n,default_flag_n
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
A,0.10342906,0.13546798,41.9922,55,406,406
B,0.09952396,0.08541667,47.7715,41,480,480
C,0.09571932,0.08771930,10.9120,10,114,114


### 1. Binomial / Jeffrey's / Hoshmer-Lemeshow / z-score

In [5]:


# Test with PDtoolkit
pp.testing(rating.label = bin_data$ratings,
           pdc = bin_data$predicted_pd_mean,
           no = bin_data$predicted_pd_n,
           nb = bin_data$default_flag_sum, 
           alpha = 0.05)

rating,no,nb,odr,pdc,alpha,binomial,binomial.res,jeffreys,jeffreys.res,zscore,zscore.res,hosmer.lemeshow,hosmer.lemeshow.res
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
A,406,55,0.13546798,0.10342906,0.05,0.02389227,H1: ODR > PDC,0.01995857,H1: ODR > PDC,0.01700479,H1: ODR > PDC,0.13025,H0: ODR <= PDC
B,480,41,0.08541667,0.09952396,0.05,0.86744061,H0: ODR <= PDC,0.84955196,H0: ODR <= PDC,0.84906675,H0: ODR <= PDC,0.13025,H0: ODR <= PDC
C,114,10,0.08771930,0.09571932,0.05,0.66055279,H0: ODR <= PDC,0.59864873,H0: ODR <= PDC,0.61421823,H0: ODR <= PDC,0.13025,H0: ODR <= PDC


### 2. Brier Score

In [6]:
library(iterativeBMA, quietly = T)

brier.score(bin_data$predicted_pd_mean, bin_data$default_flag_mean)


Attaching package: 'robustbase'


The following object is masked from 'package:survival':

    heart


Scalable Robust Estimators with High Breakdown Point (version 1.7-1)



Attaching package: 'BiocGenerics'


The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")

[1] 0.001289508

### 3. Herfindahl

In [7]:
library(creditR, quietly = T)

h <- Herfindahl.Hirschman.Index(bin_data, "default_flag_n")
h


Attaching package: 'MLmetrics'


The following object is masked from 'package:base':

    Recall



Attaching package: 'creditR'


The following object is masked from 'package:PDtoolkit':

    scaled.score




# A tibble: 3 x 10
  ratings predicted_pd_mean default_flag_mean predicted_pd_sum default_flag_sum
  <chr>               <dbl>             <dbl>            <dbl>            <int>
1 A                  0.103             0.135              42.0               55
2 B                  0.0995            0.0854             47.8               41
3 C                  0.0957            0.0877             10.9               10
# ... with 5 more variables: predicted_pd_n <int>, default_flag_n <int>,
#   SumTotal <int>, concentration <df[,1]>, HHI <df[,1]>


[1] 0.408232

### 4. Spiegelhalter

In [8]:
Spiegelhalter_z = function(y, prob){
  alpha = 0.05
  z_score = sum((y-prob)*(1-2*prob))/sqrt(sum(((1-2*prob)^2)*prob*(1-prob)))
  print(z_score)
  if (abs(z_score) > qnorm(1-alpha/2)){
    print('reject null. NOT calibrated')
  } else {
    print('fail to reject. calibrated')
  }
  cat('z score: ', z_score, '\n')
  cat('p value: ', 1-pnorm(abs(z_score)), '\n')
    
  return(z_score)
}

Spiegelhalter_z(bin_data$default_flag_mean, bin_data$predicted_pd_mean)


[1] 0.01840817
[1] "fail to reject. calibrated"
z score:  0.01840817 
p value:  0.4926566 


[1] 0.01840817

### 5. ROC calculation

In [9]:
library(pROC, quietly = T)
roc_obj <- roc(test_data_main$default_flag, test_data_main$predicted_pd)
1 - auc(roc_obj)

Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following object is masked from 'package:BiocGenerics':

    var


The following objects are masked from 'package:stats':

    cov, smooth, var


Setting levels: control = 0, case = 1

Setting direction: controls > cases



[1] 0.5008548

### 6. CLAR

In [20]:
# https://www.rdocumentation.org/packages/VUROCS/versions/1.0/topics/clar

#' @title Cumulative LGD Accuracy Ratio
#' @description Calculates for a vector of realized categories \code{y} and a vector of predicted categories \code{hx} the cumulative LGD accuarcy ratio (CLAR) according to Ozdemir and Miu 2009.
#' @param hx a vector of  predicted values.
#' @param y a vector of  realized values.
#' @return The function returns the CLAR for a vector of realized categories \code{y} and a vector of predicted categories \code{hx}.
#' @examples clar(rep(1:5, each = 3), c(3, 3, 3, rep(2:5, each = 3)))
#' @references Ozdemir, B., Miu, P., 2009. Basel II Implementation. A Guide to Developing and Validating a Compliant Internal Risk Rating System. McGraw-Hill, USA.

clar <- function(y, hx) {
    if (any(is.na(hx)) | any(is.na(y))) {
        stop("\n both 'hx' and 'y' must not contain NA values")
    }

    if (length(hx) != length(y)) {
        stop("\n both 'hx' and 'y' must be of the same length")
    }

    nx <- length(hx)
    classes <- sort(union(unique(hx), unique(y)), decreasing = TRUE)
    num <- rep(NA, length(classes))
    for (i in 1:length(classes)) {
        num[i] <- sum(hx == classes[i])
    }
    cnum <- cumsum(num)
    index <- order(hx, decreasing = TRUE)
    hx <- hx[index]
    y <- y[index]
    corr <- rep(0, length(classes))
    for (i in 1:(length(classes) - 1)) {
        if (cnum[i] > 0) corr[i] <- sum(y[1:cnum[i]] >= classes[i])
    }
    corr <- corr / nx
    corr[length(classes)] <- 1
    obs <- cnum / nx

    res <- obs[1] * corr[1] / 2
    for (i in 2:length(corr)) {
        res <- res + (corr[i] - corr[i - 1]) * (obs[i] - obs[i - 1]) / 2 + (obs[i] - obs[i - 1]) * corr[i - 1]
    }
    res * 2
}

lgd_main <- read.csv("lgd_dataset.csv")

clar(lgd_main$realised_outcome, lgd_main$predicted_outcome)

[1] 0.84

In [16]:
(rep(1:5,each=3))

[1] 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5

In [17]:
c(3,3,3,rep(2:5,each=3))

[1] 3 3 3 2 2 2 3 3 3 4 4 4 5 5 5

### 7. Loss Capture Ratio

In [ ]:
# https://github.com/selva86/InformationValue

### 8. Information Value

In [21]:
# --------------------------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See LICENSE.txt in the project root for license information.
# --------------------------------------------------------------------------------------------

#' @title
#' Calculate Weight of Evidence (WOE) and Information Value (IV) between a
#' single predictor and a single outcome variable.
#'
#' @description
#' Calculates Weight of Evidence (WOE) and Information Value (IV) between a
#' single predictor and a single outcome variable. This function implements the
#' common Information Value calculations whilst maintaining the minimum reliance
#' on external dependencies. Use `map_IV()` for the equivalent of
#' `Information::create_infotables()`, which performs calculations for multiple
#' predictors and a single outcome variable.
#'
#' @details
#' The approach used mirrors the one used in `Information::create_infotables()`.
#'
#' @param data Data frame containing the data.
#' @param outcome String containing the name of the outcome variable.
#' @param predictor String containing the name of the predictor variable.
#' @param bins Numeric value representing the number of bins to use.
#'
#' @import dplyr
#'
#' @return A data frame is returned as an output.
#'
calculate_IV <- function(data,
                         outcome,
                         predictor,
                         bins){

  pred_var <- data[[predictor]]
  outc_var <- data[[outcome]]

  # Check inputs
  if(sum(is.na(outc_var)) > 0){

    stop(
      glue::glue(
        "dependent variable {outcome} has missing values in the input training data frame"
      )
      )
  }

  # Compute q
  q <- stats::quantile(
    pred_var,
    probs = c(1:(bins - 1) / bins),
    na.rm = TRUE,
    type = 3
    )

  # Compute cuts
  cuts <- unique(q)

  # Compute intervals
  intervals <-
    findInterval(
      pred_var,
      vec = cuts,
      rightmost.closed = FALSE)

  # Compute cut_table
  cut_table <-
    table(
      intervals,
      outc_var) %>%
    as.data.frame.matrix()

  ## get min/max
  cut_table_2 <-
    data.frame(
    var = pred_var,
    intervals
  ) %>%
    group_by(intervals) %>%
    summarise(
      min = min(var, na.rm = TRUE) %>% round(digits = 1),
      max = max(var, na.rm = TRUE) %>% round(digits = 1),
      n = n(),
      .groups = "drop"
    ) %>%
    mutate(!!sym(predictor) :=
    glue::glue("[{round(min, digits = 1)},{round(max, digits = 1)}]")) %>%
    mutate(percentage = n / sum(n)) %>%
    select(!!sym(predictor), intervals, n, percentage)

  # Create variables that are double
  cut_table_1 <- as.numeric(cut_table$`1`)
  cut_table_0 <- as.numeric(cut_table$`0`)

  # Non-events in group
  n_non_event <- cut_table_1 * sum(cut_table_0) # t$y_1*sum_y_0
  n_yes_event <- cut_table_0 * sum(cut_table_1) # t$y_0*sum_y_1

  # Compute WOE

  cut_table_2$WOE <-
    ifelse(
      cut_table$`1` > 0 & cut_table$`0` > 0, # Both positive
      log(n_non_event / n_yes_event), # % of non-events divided by % of events
           0) # Otherwise impute 0

  # Compute IV_weight
  p1 <- cut_table$`1` / sum(cut_table$`1`)
  p0 <- cut_table$`0` / sum(cut_table$`0`)

  cut_table_2$IV_weight <- p1 - p0
  cut_table_2$IV <- cut_table_2$WOE * cut_table_2$IV_weight

  cut_table_2 %>%
    mutate(IV = cumsum(IV)) %>%
    # Maintain consistency with `Information::create_infotables()`
    select(
      !!sym(predictor),
      N = "n",
      Percent = "percentage",
      WOE,
      IV)
}

#' @title
#' Calculate Weight of Evidence (WOE) and Information Value (IV) between
#' multiple predictors and a single outcome variable, returning a list of
#' statistics.
#'
#' @description
#' This is a wrapper around `calculate_IV()` to loop through multiple predictors
#' and calculate their Weight of Evidence (WOE) and Information Value (IV) with
#' respect to an outcome variable.
#'
#' @details
#' The approach used mirrors the one used in `Information::create_infotables()`.
#'
#' @param data Data frame containing the data.
#' @param outcome String containing the name of the outcome variable.
#' @param predictors Character vector containing the names of the predictor
#'   variables. If `NULL` (default) is supplied, all numeric variables in the
#'   data will be used.
#' @param bins Numeric value representing the number of bins to use. Defaults to
#'   10.
#'
#' @import dplyr
#'
#' @return A list of data frames is returned as an output. The first layer of
#' the list contains `Tables` and `Summary`:
#'   -  `Tables` is a list of data frames containing the WOE and cumulative sum
#'   IV for each predictor.
#'   - `Summary` is a single data frame containing the IV for all predictors.
#'
map_IV <- function(data,
                   predictors = NULL,
                   outcome,
                   bins = 10){

  if(is.null(predictors)){

    predictors <-
      data %>%
      select(-!!sym(outcome)) %>%
      select(
        where(is.numeric)
      ) %>%
      names()
  }

  # List of individual tables
  Tables <-
    predictors %>%
    purrr::map(function(pred){

      calculate_IV(
        data = data,
        outcome = outcome,
        predictor = pred,
        bins = bins
      )
    }) %>%
    purrr::set_names(
      nm = purrr::map(
        .,
        function(df){
          names(df)[[1]]
        }
      )
    )

  # Compile Summary Table
  Summary <-
    list("df" = Tables,
         "names" = names(Tables)) %>%
    purrr::pmap(function(df, names){

      IV_final <-
        df %>%
        slice(nrow(df)) %>%
        pull(IV)

      data.frame(
        Variable = names,
        IV = IV_final
      )
    }) %>%
    bind_rows() %>%
    arrange(desc(IV))

  # Reorder and combine list
  c(
    list("Tables" = Tables[Summary$Variable]), # Reordered
    list("Summary" = Summary)
  )
}


pd_german_data <- read.csv("german_data.csv")


In [37]:
res = map_IV (pd_german_data,
              predictors = c('amount','duration'),
              outcome='GoodCredit',
              bins = 10)

In [38]:
res$Summary

Variable,IV
<chr>,<dbl>
duration,0.2778772
amount,0.1117618


In [42]:
q = calculate_IV(pd_german_data,
                predictor='amount',
                outcome='GoodCredit',
                bins = 10)

In [46]:
q

amount,N,Percent,WOE,IV
<glue>,<int>,<dbl>,<dbl>,<dbl>
"[250,931]",99,0.099,0.06177736,0.0003824313
"[932,1258]",99,0.099,0.01438874,0.0004029866
"[1262,1474]",99,0.099,-0.23789141,0.0057272229
"[1478,1901]",102,0.102,-0.38665578,0.0197204795
"[1905,2315]",100,0.100,-0.04808619,0.0199494614
"[2319,2835]",100,0.100,-0.25131443,0.0259331383
"[2848,3578]",100,0.100,-0.36101335,0.0379669164
"[3590,4712]",100,0.100,-0.04808619,0.0381958983
"[4716,7166]",100,0.100,0.31508105,0.0486985998


In [47]:
sum(q$IV)

[1] 0.3087389

In [23]:
pd_german_data

GoodCredit,checkingstatus,duration,history,purpose,amount,savings,employ,installment,status,⋯,residence,property,age,otherplans,housing,cards,job,liable,tele,foreign
<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,⋯,<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
0,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,2,A121,22,A143,A152,1,A173,1,A191,A201
0,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,3,A121,49,A143,A152,1,A172,2,A191,A201
0,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,4,A122,45,A143,A153,1,A173,2,A191,A201
1,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,4,A124,53,A143,A153,2,A173,2,A191,A201
0,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,4,A124,35,A143,A153,1,A172,2,A192,A201
0,A14,24,A32,A42,2835,A63,A75,3,A93,⋯,4,A122,53,A143,A152,1,A173,1,A191,A201
0,A12,36,A32,A41,6948,A61,A73,2,A93,⋯,2,A123,35,A143,A151,1,A174,1,A192,A201
0,A14,12,A32,A43,3059,A64,A74,2,A91,⋯,4,A121,61,A143,A152,1,A172,1,A191,A201


### 9. LGD t-test

### 10. Migration matrix stability

### TO BE DELETED

In [10]:
brier.score

function (predictedArr, truthArr) 
{
    if (length(predictedArr) != length(truthArr)) {
        print("ERROR: length NOT equal!!")
    }
    temp.vec <- (truthArr - predictedArr)^2
    sum(temp.vec)
}
<bytecode: 0x00000000330ca708>
<environment: namespace:iterativeBMA>

In [11]:
pp.testing

function (rating.label, pdc, no, nb, alpha = 0.05) 
{
    l <- list(rating.label, pdc, no, nb)
    l.check <- !length(table(sapply(l, length))) == 1
    if (l.check) {
        stop("arguments rating.label, pdc, no and nb have to be of the same length.")
    }
    if (any(!(is.numeric(pdc) | is.numeric(n) | is.numeric(nb)))) {
        stop("All arguments have to of numeric type.")
    }
    if (any(pdc > 1 | pdc < 0)) {
        stop("pdc and odr have to be between 0 and 1.")
    }
    if (any(nb > no)) {
        stop("Number of defaults cannot be greater than number of observations.")
    }
    if (any(nb < 0 | no < 0)) {
        stop("no and nb arguments cannot be negative.")
    }
    if (alpha > 1 | alpha < 0) {
        stop("significance level (alpha) has to be between 0 and 1.")
    }
    bino.test <- pbinom(nb - 1, no, pdc, lower.tail = FALSE)
    bino.test.res <- ifelse(bino.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    jeff.test <- pbeta(pdc, nb + 0.5, no - nb + 0.5)
    jeff.test.res <- ifelse(jeff.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    odr <- nb/no
    zsco.test <- zscore.test(pdc = pdc, odr = odr, no = no)
    zsco.test.res <- ifelse(zsco.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    if (length(pdc) == 1) {
        hosm.test <- NA
        hosm.test.res <- "Only one rating grade."
    }
    else {
        hosm.test <- hl.test(pdc = pdc, no = no, nb = nb)
        hosm.test.res <- ifelse(hosm.test >= alpha, "H0: ODR <= PDC", 
            "H1: ODR > PDC")
    }
    res <- data.frame(rating = rating.label, no = no, nb = nb, 
        odr = odr, pdc = pdc, alpha = alpha, binomial = bino.test, 
        binomial.res = bino.test.res, jeffreys = jeff.test, jeffreys.res = jeff.test.res, 
        zscore = zsco.test, zscore.res = zsco.test.res, hosmer.lemeshow = hosm.test, 
        hosmer.lemeshow.res = hosm.test.res)
    return(res)
}
<bytecode: 0x000000002ba1f778>
<environment: namespace:PDtoolkit>

In [12]:
hl.test <- function(pdc, no, nb) {
	k <- length(no)
	hl <- sum((nb - no * pdc)^2 / (no * pdc * (1 - pdc)), na.rm = T)
	p.val <- pchisq(q = hl, df = k, lower.tail = FALSE)
return(p.val)
}